# Regrid CESM-SMYLE with xESMF
Sam Mogen

In [1]:
# packages
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import matplotlib.pyplot as plt
%matplotlib inline

# SMYLE Utility functions
from SMYLEutils import io_utils as io
from SMYLEutils import calendar_utils as cal
from SMYLEutils import stat_utils as stat

In [32]:
var = 'omega_arag'

# run for all inits
init = '11' # '02','05', '08', '11'

In [33]:
omega = xr.open_dataset(var + '.monthly.' + init + '.nc')

## Regrid and save out!

In [34]:
import xesmf as xe

In [35]:
obs = xr.open_dataset('/glade/work/smogen/SMYLE-personal/OceanSODA-ETHZ_1985-2019_v2020b.regrid2.nc')

In [36]:
regridder_smyle = xe.Regridder(omega, obs, 'bilinear', periodic=True)

In [37]:
%%time
smyle_seas_rg = regridder_smyle(omega)

/glade/u/home/smogen/.conda/envs/smyle-analysis/lib/python3.8/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CPU times: user 1min 26s, sys: 5.99 s, total: 1min 32s
Wall time: 1min 38s


In [38]:
# size of the dataset
print(smyle_seas_rg.nbytes / 1e9) # GB
print(omega.nbytes / 1e9) # GB

12.192773068
11.562517228


In [39]:
# FOSI Regrid
smyle_seas_rg.to_netcdf(var + init + '.regrid.nc')

In [40]:
del smyle_seas_rg, omega